In [1]:
import pandas as pd
import numpy as np

# 1 Считаем датасет из файла
df = pd.read_csv('train.csv')

# 2 Выведем основную информацию о датасете
print("\nПервые 5 строк:")
print(df.head())

print("\nИнформация о типах данных и пропусках:")
print(df.info())

print("\nОписательная статистика:")
print(df.describe())

print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())


Первые 5 строк:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   

In [3]:
# 3 Посчитаем процент выживаемости у каждого класса пассажиров
print("3. ПРОЦЕНТ ВЫЖИВАЕМОСТИ ПО КЛАССАМ:")
survival_by_class = df.groupby('Pclass')['Survived'].agg(['mean', 'count'])
survival_by_class['survival_rate'] = (survival_by_class['mean'] * 100).round(2)
print(survival_by_class[['survival_rate', 'count']])

3. ПРОЦЕНТ ВЫЖИВАЕМОСТИ ПО КЛАССАМ:
        survival_rate  count
Pclass                      
1               62.96    216
2               47.28    184
3               24.24    491


In [20]:
# 4 Функция для извлечения нормального имени
import re
def extract_first_name(full_name: str) -> str:

# Функция extract_first_name(full_name)

# Назначение:
#     Извлекает имя пассажира из поля "Name" датасета Titanic.

# Описание:
#     Если в строке есть скобки — берётся первое слово внутри них (обычно женское имя).
#     Если скобок нет — берётся первое слово после титула (после точки).

    # 1 сначала пробуем вытащить имя из скобок (для женщин типа "Mrs. ... (Florence ...)")
    in_parentheses = re.findall(r"\(([^)]+)\)", full_name)
    if in_parentheses:
        # берём первое слово из скобок
        return in_parentheses[0].split()[0]

    # 2 если скобок нет — берём то, что после титула
    # пример: "Braund, Mr. Owen Harris"
    # после запятой -> " Mr. Owen Harris"
    after_comma = full_name.split(",")[1].strip()          # "Mr. Owen Harris"
    after_title = after_comma.split(".")[1].strip()        # "Owen Harris"
    first_name = after_title.split()[0]                    # "Owen"
    return first_name


# применяем к датафрейму
df["FirstName"] = df["Name"].apply(extract_first_name)

# 4 Самое популярное мужское и женское имя на корабле -----------------------
male_names = df[df["Sex"] == "male"]["FirstName"].value_counts()
female_names = df[df["Sex"] == "female"]["FirstName"].value_counts()

most_popular_male = male_names.index[0] if not male_names.empty else None
most_popular_female = female_names.index[0] if not female_names.empty else None

print("\nСамые популярные имена на корабле")
print("Мужское:", most_popular_male)
print("Женское:", most_popular_female)


Самые популярные имена на корабле
Мужское: William
Женское: Anna


In [28]:
# 5 Самое популярное мужское и женское имя в каждом классе
print("\nСамые популярные мужские и женские имена по классам")
for cls in sorted(df["Pclass"].unique()):
    males_cls = df[(df["Pclass"] == cls) & (df["Sex"] == "male")]["FirstName"].value_counts()
    females_cls = df[(df["Pclass"] == cls) & (df["Sex"] == "female")]["FirstName"].value_counts()

    top_male = males_cls.index[0] if not males_cls.empty else "—"
    top_female = females_cls.index[0] if not females_cls.empty else "—"

    print(f"\nКласс {cls}:")
    print("  самое популярное мужское имя :", top_male)
    print("  самое популярное женское имя :", top_female)


Самые популярные мужские и женские имена по классам

Класс 1:
  самое популярное мужское имя : William
  самое популярное женское имя : Elizabeth

Класс 2:
  самое популярное мужское имя : William
  самое популярное женское имя : Elizabeth

Класс 3:
  самое популярное мужское имя : William
  самое популярное женское имя : Anna


In [26]:
# 6 Таблица с пассажирами, возраст которых > 44
older_44 = df[df["Age"] > 44]
print("\nПассажиры старше 44 лет (первые строки)")
print(older_44.head(10))


Пассажиры старше 44 лет (первые строки)
    PassengerId  Survived  Pclass                                      Name  \
6             7         0       1                   McCarthy, Mr. Timothy J   
11           12         1       1                  Bonnell, Miss. Elizabeth   
15           16         1       2          Hewlett, Mrs. (Mary D Kingcome)    
33           34         0       2                     Wheadon, Mr. Edward H   
52           53         1       1  Harper, Mrs. Henry Sleeper (Myna Haxtun)   
54           55         0       1            Ostby, Mr. Engelhart Cornelius   
62           63         0       1               Harris, Mr. Henry Birkhardt   
92           93         0       1               Chaffee, Mr. Herbert Fuller   
94           95         0       3                         Coxon, Mr. Daniel   
96           97         0       1                 Goldschmidt, Mr. George B   

       Sex   Age  SibSp  Parch       Ticket     Fare Cabin Embarked  FirstName  
6     ma

In [27]:
# 7. Пассажиры мужского пола, возраст < 44
younger_males = df[(df["Age"] < 44) & (df["Sex"] == "male")]
print("\nМужчины младше 44 лет (первые строки)")
print(younger_males.head(10))


Мужчины младше 44 лет (первые строки)
    PassengerId  Survived  Pclass                            Name   Sex   Age  \
0             1         0       3         Braund, Mr. Owen Harris  male  22.0   
4             5         0       3        Allen, Mr. William Henry  male  35.0   
7             8         0       3  Palsson, Master. Gosta Leonard  male   2.0   
12           13         0       3  Saundercock, Mr. William Henry  male  20.0   
13           14         0       3     Andersson, Mr. Anders Johan  male  39.0   
16           17         0       3            Rice, Master. Eugene  male   2.0   
20           21         0       2            Fynney, Mr. Joseph J  male  35.0   
21           22         1       2           Beesley, Mr. Lawrence  male  34.0   
23           24         1       1    Sloper, Mr. William Thompson  male  28.0   
27           28         0       1  Fortune, Mr. Charles Alexander  male  19.0   

    SibSp  Parch     Ticket     Fare        Cabin Embarked FirstName 

In [25]:
# 8 Количества n-местных кают (2,3,4)
# из выведенных таблиц видно, что у пассажира может быть в поле Cabin записано
# несколько номеров, например C23 C25 C27. Разделим их

cabins_df = df.dropna(subset=["Cabin"]).copy()
cabins_df["CabinT"] = cabins_df["Cabin"].str.split()
exploded = cabins_df.explode("CabinT")

occupancy_per_cabin = exploded["CabinT"].value_counts()

# Сколько кают имели 1, 2, 3, ... пассажиров
n_place_counts = occupancy_per_cabin.value_counts().sort_index()

print("\nКоличество n-местных кают:")
for n, cnt in n_place_counts.items():
    print(f"{n}-местных кают: {cnt}")

print("\nОтвет:")
answer = ''
for n, cnt in n_place_counts.items():
    if n>=2:
      answer += f"{n}-местных кают: {cnt}. "
print(answer)


Количество n-местных кают:
1-местных кают: 104
2-местных кают: 44
3-местных кают: 6
4-местных кают: 7

Ответ:
2-местных кают: 44. 3-местных кают: 6. 4-местных кают: 7. 
